---
title: "from raw to identification"
author: "Xiaodong Feng"
date: '`r format(Sys.time(), "%d %B, %Y")`'
output:
  BiocStyle::html_document:
    toc: true
    toc_depth: 2
    number_sections: true
    toc_float: true

vignette: >
  %\VignetteEngine{knitr::rmarkdown}
  %\VignetteEncoding{UTF-8}
---

In [ ]:
knitr::opts_chunk$set(echo = TRUE)
knitr::opts_knit$set(root.dir = "d:/github/DynamicTol/vignettes") # The user needs to change the directory accordingly

# Introduction

This document contains basic steps of processing LC-MS datasets. This workflow aims to improve the identification part in LC-MS-based metabolomics. 

# Load required packages and functions
The user needs to change the directory accordingly

In [ ]:
source("d:/github/DynamicTol/vignettes/Alternative way to use the package.R") 

# Peak picking of Pure internal standards in LTQ Orbitrap XL based on the xcms package

## Import data

In this example, we use the pure internal standards containing 15 internal standards, which was generated by LTQ Orbitrap XL, see details in the R package of *RMassBankData*. The .raw dataset were converted into .mzML format using the msconvert package, with the centroid method selected. The .mzML format dataset were then be loaded into XCMS using readMSData command.

In [ ]:
## Get the full path to the .mzml files
Mzml <- dir(system.file("spectra", package = "RMassBankData"), 
            full.names = TRUE, recursive = TRUE)
BaseName <- sub(basename(Mzml),pattern = ".mzML",replacement = "", fixed = TRUE)
pd <- data.frame(
  sample_name = BaseName ,
  sample_group = BaseName, stringsAsFactors = FALSE
)
onDisk <- readMSData(files = Mzml, pdata = new("NAnnotatedDataFrame", pd), mode = "onDisk")
head(fData(onDisk))
table(msLevel(onDisk))

# Peak detection

In this example, we perform the peak detection using XCMS's Centwave aligorithm. The parameters used in peak detection are important. The user need to adjust these parameters according to their own dataset. Here we perform the dynamic binning peak detection as described in *X. Feng, W. Zhang, F. Kuipers, I. Kema, A. Barcaru, P. Horvatovich, Dynamic binning peak detection and assessment of various lipidomics liquid chromatography-mass spectrometry pre-processing platforms suppl. Anal. Chim. Acta. 31, 1–17 (2021)* We first need to calculate the proper constant A according to mass resoving power and reference m/z. The specificities are based on *https://biotech.ufl.edu/portfolio/ltq-orbitrap-xl-system/*

In [ ]:
MRP <- 60000; MRPValue <- 60000;# 
RefMZ <- 400; RefMZValue <- 400; #  Velos is mz reference 400 while Q-Exactive is 200.
AValue <- 1 / (MRP * (RefMZ^0.5) * 2.35482)
print(AValue)
cwp <- CentWaveParam(
  A = AValue, ppm = 3, Instrument = 2,
  peakwidth = c(2.4, 30), snthresh = 5, noise = 100000, prefilter = c(1, 200000),
  firstBaselineCheck = FALSE, integrate = 2
)
xdata <- xcms::findChromPeaks(onDisk, param = cwp)
head(chromPeaks(xdata))

Let's check the results after peak detection, the selected retention time of 
222 seconds and m/z value of 136.1121 Da is based on the value of Amphetamine

In [ ]:
rtSel <- 222
mzSel <- 136.1121
pks <- data.frame(peak_id = rownames(chromPeaks(xdata)), chromPeaks(xdata))
pks_DT <- data.table::data.table(pks)
pks_DT[abs(mz-mzSel) < 0.02 & abs(rt-rtSel) < 10]

# MS1 Peak annotations based on the MetaboAnnotation package 

At this step we aim to annote the MS1 peak based on the the Internal standards information 
described in the *RMassBankData*. Thus, the std_ions.txt contains the information 
of the 15 internal standards accordingly. Especially the retention time, 
which is critical for accurate identification. However, there are limited retention time records in the publice database. It is also worth to stress that the output collumns of 'score' and 'score_rt" are 'mass difference in Da' and  'retention time difference', respectively. To avoid confusion, we changed the names accordingly. Notice, the user needs to change the dir according to where the package is located.

In [ ]:
std_ions <- read.table("d:/github/DynamicTol/input/txt/std_ions.txt", 
                       sep = "\t", header = TRUE)
head(std_ions)
parm <- MetaboAnnotation::Mass2MzRtParam(adducts = c("[M+H]+", "[M+Na]+", "[M+K]+"),toleranceRt = 10)
pks_match <- MetaboAnnotation::matchValues(pks, std_ions, parm,rtColname = c("rt", "ion_rt"))
pks_match
pks_match_DT <- as.data.table(MetaboAnnotation::matchedData(pks_match, colnames(pks_match)))
head(pks_match_DT)
names(pks_match_DT)
setnames(pks_match_DT,c('score','score_rt'),c('da_error','rt_error'))
names(pks_match_DT)

# MS2 peaks export based on the Spectra package 

As we want to calculate the MS2 similarity, we have to export the MS2 peaks
here we use the functions from the *Spectra* package. In detail, we export the 
related MS2 according to the index in MS1. 
Some detected MS1 may not contain the related MSMS, we exclude them in this step.

In [ ]:
dda_spectra <- chromPeakSpectra(xdata, return.type = "Spectra", peaks = pks_match$peak_id)
dda_spectra
unique(dda_spectra$peak_id)
register(SerialParam()) # disable paralell
## Merge the spectra to obtain the consensus spectrum
dda_spectrum <- Spectra::combineSpectra(dda_spectra, FUN = combinePeaks, ppm = 20,
                              peaks = "intersect", minProp = 0.8,
                              intensityFun = median, mzFun = median,
                              f = dda_spectra$peak_id)
dda_spectrum
## Normalization to 100 according to maximum
Spectra::plotSpectra(dda_spectrum[1]) # before normalization
dda_spectrum <- addProcessing(dda_spectrum, FUN = norm_fun)
Spectra::plotSpectra(dda_spectrum[1]) # after normalization
## Extract the MS2 peaks
dda_spectrum_peaks <- peaksData(dda_spectrum)
dda_spectrum_peaks[[1]]
## Combine the peaks and add the meta id
peaks.rbind <- data.table()
for (id in c(1:length(dda_spectrum_peaks))) {
  # print(id)
  peak <- data.table(dda_spectrum_peaks[[id]]) %>% .[,library_spectra_meta_id:=id]
  peaks.rbind <- rbind(peaks.rbind, peak)
}
setnames(peaks.rbind, 'intensity', 'i')
dda_spectrum_DT <- as.data.table(spectraData(dda_spectrum))

# Creat query.sqlite database 

Current workflow supports the library search based on the .sqlite database files,
Thus we need to transfer the results from previous steps into the .sqlite formats.

In [ ]:
con_q <- DBI::dbConnect(RSQLite::SQLite(),'d:/github/DynamicTol/input/sqlite/Narcotics.sqlite')
## library_spectra_meta
spectra_meta_q <- left_join(dda_spectrum_DT,pks_match_DT,by= 'peak_id') %>%
  cbind(.,id=c(1:nrow(.)))
names(spectra_meta_q)
setnames(
  spectra_meta_q, c("precursorMz", "rtime", "target_Name", "collisionEnergy", "target_compound_id", "target_ion_adduct", "target_INCHIKEY", "target_SMILES"),
  c("precursor_mz", "retention_time", "name", "collision_energy", "accession", "precursor_type", "inchikey_id", "smiles")
)
names(spectra_meta_q)
DBI::dbWriteTable(con_q, name='library_spectra_meta', value=spectra_meta_q, overwrite=TRUE) # we need to stick to this name of library_spectra_meta, otherwise will get error
## library_spectra
DBI::dbWriteTable(con_q, name='library_spectra', value=peaks.rbind, overwrite=TRUE)

# Creat library .sqlite database

To perform the spectral matching, we first need to prepare the library. Here we downloaded the library from [msp2db](https://github.com/computational-metabolomics/msp2db/releases/tag/v0.0.14-mona-23042021), which includes the database created by parsing all of the available MoNA MSP files as of 23/04/2021. As the example dataset is in positive mode, so here we only use the positive library with the parameter *l_accessionsValue*. As the database is quite large, thus it is not included in the package, the user needs to download the database by themselves and change the working dir accordingly.

In [ ]:
l_dbPthValue <- "d:/github/DynamicTol/input/sqlite/20210423_mona.sqlite"
con <- DBI::dbConnect(RSQLite::SQLite(), l_dbPthValue)
library_spectra_meta <- con %>%
  dplyr::tbl("library_spectra_meta") %>%
  dplyr::collect() %>%
  as.data.table(.)
## Create the library accessions, to reduce the library
unique(library_spectra_meta$instrument_type)
MetaPos <- library_spectra_meta[polarity == "positive" & ms_level == 2 &
  instrument_type %in% c("LC-ESI-ITFT", "LC-ESI-QFT", "ESI-QFT", NA)] # only use the orbitrap dataset
unique(MetaPos$instrument_type)
l_accessionsValue <- MetaPos$accession
l_accessionsValue[1]
length(l_accessionsValue) # 48800

# Search query .sqlite against library .sqlite
With the query and library database, we can start spectral matching. As the example dataset is in positive mode, we only used a subset of library with positve mode. We also used a subset of XCMS results to reduce the library searching time. The user may need the 'memory.limit(size=56000)' command to increase the working memory. The query database was created in the previous step, with the q_dbPthValue to indicate the working directroy. The library database was indicated by the l_dbPthValue parameter. 
The searching module is modified based on the the *spectralMatching.R* file in the *msPurity* package, with the following novelties:
-   Implement the dynamic theory into the peak matching part, original is 10 ppm as default. Accurate identification relies heavily on MS2 alignment. For low-resolution mass spectrometry, the peak binning method was used, represented by default 1 Da value in Msnbase; While for high-resolution mass spectrometry, the peak matching method was proposed and applied in OrgMassSpecR and msPurity. The peak matching method is usually based on a fixed tolerance, such as 0.25 Da of the default setting in OrgMassSpecR, 0.005 Da for Orbitrap, and 0.015 Da for QTOF. However, a fixed tolerance in the peak matching method may cause misalignment, as the mass resolution will change according to the mz value. In our work, a dynamic mass tolerance was set for peak matching method according to the mass spectrometry types, which is proportional to 〖mz〗\^2 for FTICR, to 〖mz〗\^1.5 for Orbitrap, to m/z for Q-TOF and is a constant for Quadrupole mass analyzer, respectively. 
-   Implement the decoy searching with dynamic shift.
-   Calculation of Xcorr based on target and decoy dot-product score.
-   Calculation of Pep score based on the distribution target and decoy dot-product score.
In order to speed up the library search, we discard the *align* function and replace it with *F_DynamicMatching* function. This function take the mass resolving power (MRP) and reference m/z (RefMZ) into consideration to calculate the dynamic peak matching mass tolerance (PMMT). See details in the paper *Improving spectral library search untargeted metabolomics identification by dynamic tolerance peak matching and false discovery estimation*.
Now let's start the library search, to save the computational time, here we only used one example with the command of *spectra_meta_q$id[4]*, and used the tictoc package to record the processing time, the total processing time of this library search took 64.3 seconds. The library searching results are recorded below.

In [ ]:
tictoc::tic() # Starting time of search
Pth_Narcotics <- "d:/github/DynamicTol/input/sqlite/Narcotics.sqlite"
Pth_Mona <- "d:/github/DynamicTol/input/sqlite/20210423_mona.sqlite"
# for test begin
for (id in spectra_meta_q$id[4]) {
  SingleResult <- spectralMatching(q_dbPth = Pth_Narcotics, l_dbPth = Pth_Mona, l_accessions = l_accessionsValue, q_pids = id, mztol = NA)
}
SingleResult
# for test end
tictoc::toc() # Ending time of search 64.3 sec elapsed


We can then have a look of the matched results, among them 
\* dpc - dot product cosine of the match 
\* dpc.decoy - dot product cosine of the match calculated based on the decoy database 
\* dpc.xcorr - cross correlation score of dpc 
\* library_rt - retention time of library spectra 
\* library_accession - library accession value (unique string or number given to either MoNA or Massbank data entires) 
\* library_precursor_mz - library precursor mz 
\* library_precursor_type - library precursor type (i.e. adduct) 
\* library_entry_name - Name given to the library spectra 
\* library_inchikey - inchikey of the matched library spectra 
\* library_lpid - id in database of library spectra 
\* query_qpid - id in database of query spectra 
\* query_rt - retention time of query spectra 
\* query_precursor_mz - query precursor mz 
\* query_inchikey - inchikey of the query spectra 


# Session information

In [ ]:
sessionInfo() 